In [ ]:
import requests
import base64
import json
import pandas
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark import Session
from datetime import datetime, timedelta
import pytz


In [ ]:
# get session
session = get_active_session()

In [ ]:
-- change to working schema
use schema python_workloads.genesys

In [ ]:
# Get Genesys ID and Secret
from snowflake.snowpark.context import get_active_session

# Snowflake session initialization
session = get_active_session()

client_key_query = "select python_workloads.genesys.get_client_key()"
client_key = session.sql(client_key_query).collect()[0][0]


client_secret_query = "select python_workloads.genesys.get_client_secret()"
client_secret = session.sql(client_secret_query).collect()[0][0]


In [ ]:
# https://developer.genesys.cloud/authorization/platform-auth/base-64-encoding

encodedData = base64.b64encode(bytes(f"{client_key}:{client_secret}", "ISO-8859-1")).decode("ascii")
authorization_header_string = f"Authorization: Basic {encodedData}"
print(authorization_header_string)
print('client_key : ', client_key)
print('client_secret : ', client_secret)

In [ ]:
# Get Bearer Token

# https://developer.genesys.cloud/authorization/platform-auth/use-client-credentials

token_endpoint = 'https://login.mypurecloud.com/oauth/token'

headers = {"Content-Type":"application/x-www-form-urlencoded",
           "Authorization": "Basic {0}".format(encodedData)
          
}


post_data = {
    "grant_type" : "client_credentials"
}

# print(headers)

response = requests.post(token_endpoint, data=post_data, headers=headers)

print(response.text)
print(response.content)
response_dic=response.json()
auth_token=response_dic['access_token']
print(auth_token)

In [ ]:
# create fetch function
def fetch_data(url, token, query_payload=False):
    api_endpoint=url 
    headers = {"Content-Type":"application/json",
           "Authorization": "Bearer {0}".format(token)
          }
    if query_payload:
        response=requests.post(api_endpoint, headers=headers, data=json.dumps(query_payload))
    else:
        response=requests.get(api_endpoint, headers=headers)
    response_json=response.json()
    return response_json

In [ ]:
# license api

# snowflake_df write mode "append” will: Append data of this DataFrame to the existing table. Creates a table if it does not exist.
table_name='genesys.raw.license_json'

# initial endpoint
page_size=100
page_number=1
base_url='https://api.mypurecloud.com'
endpoint='/api/v2/license/users?pageSize={0}&pageNumber={1}'.format(page_size, page_number)
api_endpoint = '{0}{1}'.format(base_url, endpoint)
print(api_endpoint, auth_token)

# flag to stop pagination
keep_paginating=True 

# list to collect all data from each pagination api hit
all_data=[]

# pagination loop
while keep_paginating:
    users_data=fetch_data(api_endpoint, auth_token)
    all_data.extend(users_data['entities'])
    print(users_data)
    print('entity keys ',users_data['entities'][0].keys())
    if 'nextUri' in users_data.keys():
        api_endpoint = '{0}{1}'.format(base_url, users_data['nextUri'])
    else:
        keep_paginating=False  

# log results
print(all_data)
print('Record count: ', len(all_data))

#create df with results
df=pandas.DataFrame({'json_data':all_data})
snowflake_df=session.create_dataframe(df)

# truncate staging table and load data. comment out for first run
session.sql(f"truncate table {table_name}").collect() 
# snowflake_df write mode "append” will: Append data of this DataFrame to the existing table. Creates a table if it does not exist.
snowflake_df.write.mode("append").save_as_table(table_name)

In [ ]:
create or replace table genesys.transform.licenses as
select 
    base."json_data" as json_data
    , base."json_data":id::varchar as userid
    , licenses.value::varchar as licenses
    , base."json_data":selfUri::varchar as selfUri
from 
    genesys.raw.license_json as base,
    lateral flatten (input=>base."json_data":licenses) as licenses

In [ ]:
# license api

# snowflake_df write mode "append” will: Append data of this DataFrame to the existing table. Creates a table if it does not exist.
table_name='genesys.raw.lookup_roles_json'

# initial endpoint
page_size=100
page_number=1
base_url='https://api.mypurecloud.com'
endpoint='/api/v2/authorization/roles?pageSize={0}&pageNumber={1}'.format(page_size, page_number)
api_endpoint = '{0}{1}'.format(base_url, endpoint)
print(api_endpoint, auth_token)

# flag to stop pagination
keep_paginating=True 

# list to collect all data from each pagination api hit
all_data=[]

# pagination loop
while keep_paginating:
    users_data=fetch_data(api_endpoint, auth_token)
    print(users_data)
    all_data.extend(users_data['entities'])
    print(users_data)
    print('entity keys ',users_data['entities'][0].keys())
    if 'nextUri' in users_data.keys():
        api_endpoint = '{0}{1}'.format(base_url, users_data['nextUri'])
    else:
        keep_paginating=False  

# log results
print(all_data)
print('Record count: ', len(all_data))

#create df with results
df=pandas.DataFrame({'json_data':all_data})
snowflake_df=session.create_dataframe(df)

# truncate staging table and load data. comment out for first run
session.sql(f"truncate table {table_name}").collect() 
# snowflake_df write mode "append” will: Append data of this DataFrame to the existing table. Creates a table if it does not exist.
snowflake_df.write.mode("append").save_as_table(table_name)

In [ ]:
create or replace table genesys.transform.lookup_roles as
select 
    base."json_data" as json_data
    , base."json_data":id::varchar as id
    , base."json_data":name::varchar as role_name
    , base."json_data":description::varchar as role_description
    , iff( base."json_data":default=true, 'Default', 'Custom')::varchar as role_type
    , base."json_data":permissions::varchar as permissions
    , base."json_data":userCount::varchar as user_count
from 
    genesys.raw.lookup_roles_json as base

In [ ]:
# stations api

# snowflake_df write mode "append” will: Append data of this DataFrame to the existing table. Creates a table if it does not exist.
table_name='genesys.raw.stations_json'

# initial endpoint
page_size=100
page_number=1
base_url='https://api.mypurecloud.com'
endpoint='/api/v2/stations?pageSize={0}&pageNumber={1}'.format(page_size, page_number)
api_endpoint = '{0}{1}'.format(base_url, endpoint)

# flag to stop pagination
keep_paginating=True 

# list to collect all data from each pagination api hit
all_data=[]

# pagination loop
while keep_paginating:
    users_data=fetch_data(api_endpoint, auth_token)
    all_data.extend(users_data['entities'])
    print(users_data)
    print('entity keys ',users_data['entities'][0].keys())
    if 'nextUri' in users_data.keys():
        api_endpoint = '{0}{1}'.format(base_url, users_data['nextUri'])
    else:
        keep_paginating=False

# log results
print(all_data)
print('Record count: ', len(all_data))

#create df with results
df=pandas.DataFrame({'json_data':all_data})
snowflake_df=session.create_dataframe(df)

# truncate staging table and load data. comment out for first run
session.sql(f"truncate table {table_name}").collect() 
# snowflake_df write mode "append” will: Append data of this DataFrame to the existing table. Creates a table if it does not exist.
snowflake_df.write.mode("append").save_as_table(table_name)

In [ ]:
create or replace table genesys.transform.stations as
select 
    base."json_data" as json_data
    , base."json_data":id::varchar as id
    , base."json_data":selfUri::varchar as selfUri
    , base."json_data":userId::varchar as userId
    , base."json_data":webRtcUserId::varchar as webRtcUserId
    , base."json_data":name::varchar as name
    , split_part(base."json_data":name::varchar,' - ', 0) as fullName
    , split_part(fullName,' ', 0) as firstName
    , split_part(fullName,' ', -1) as lastName -- "-1" to account for middle name(s)
    , base."json_data":status::varchar as status
    , base."json_data":type::varchar as type
    , base."json_data":lineAppearanceId::varchar as lineAppearanceId
from 
    genesys.raw.stations_json as base

In [ ]:
# stations api

# snowflake_df write mode "append” will: Append data of this DataFrame to the existing table. Creates a table if it does not exist.
table_name='genesys.raw.lookup_locations_json'

# initial endpoint
page_size=100
page_number=1
base_url='https://api.mypurecloud.com'
endpoint='/api/v2/locations?pageSize={0}&pageNumber={1}'.format(page_size, page_number)
api_endpoint = '{0}{1}'.format(base_url, endpoint)

# flag to stop pagination
keep_paginating=True 

# list to collect all data from each pagination api hit
all_data=[]

# pagination loop
while keep_paginating:
    users_data=fetch_data(api_endpoint, auth_token)
    all_data.extend(users_data['entities'])
    print('entity keys ',users_data['entities'][0].keys())
    if 'nextUri' in users_data.keys():
        api_endpoint = '{0}{1}'.format(base_url, users_data['nextUri'])
    else:
        keep_paginating=False

# log results
print(all_data)
print('Record count: ', len(all_data))

#create df with results
df=pandas.DataFrame({'json_data':all_data})
snowflake_df=session.create_dataframe(df)

# truncate staging table and load data. comment out for first run
session.sql(f"truncate table {table_name}").collect() 
# snowflake_df write mode "append” will: Append data of this DataFrame to the existing table. Creates a table if it does not exist.
snowflake_df.write.mode("append").save_as_table(table_name)

In [ ]:
create or replace table genesys.transform.lookup_locations as
select 
    base."json_data" as json_data
    , base."json_data":id::varchar as id
    , base."json_data":contactUser.id::varchar as userId
    , base."json_data":contactUser.selfUri::varchar as selfUri
from 
    genesys.raw.lookup_locations_json as base

In [ ]:
-- query for distinct list of userIDs
select
    agents.userid
from
    genesys.transform.agents
group by 
    all

In [ ]:
# Convert and format the retreived data into a dataframe
userIds_df = get_unique_userIds.to_pandas()

# print(userIds_df)

In [ ]:
# roles api

# snowflake_df write mode "append” will: Append data of this DataFrame to the existing table. Creates a table if it does not exist.
table_name='genesys.raw.locations_json'

# initial endpoint
for index, row in userIds_df.iterrows():
    print(row['USERID'])
    print(client_key)

    # create endpoint URL with USERID, print
    base_url='https://api.mypurecloud.com'
    endpoint='/api/v2/users/{0}/geolocations/{1}'.format(row['USERID'], client_key)
    api_endpoint = '{0}{1}'.format(base_url, endpoint)
    print(api_endpoint, auth_token)

    # call fetch_data, passing endpoint and auth token
    users_data=fetch_data(api_endpoint, auth_token)

    print(users_data)

# list to collect all data from each pagination api hit
all_data=[]

# log results
print(all_data)
print('Record count: ', len(all_data))

#create df with results
df=pandas.DataFrame({'json_data':all_data})
snowflake_df=session.create_dataframe(df)

# truncate staging table and load data. comment out for first run
session.sql(f"truncate table {table_name}").collect() 
# snowflake_df write mode "append” will: Append data of this DataFrame to the existing table. Creates a table if it does not exist.
snowflake_df.write.mode("append").save_as_table(table_name)

In [ ]:
# roles api

# snowflake_df write mode "append” will: Append data of this DataFrame to the existing table. Creates a table if it does not exist.
table_name='genesys.raw.roles_json'

# initial endpoint
for index, row in userIds_df.iterrows():
    print(row['USERID'])

    # create endpoint URL with USERID, print
    base_url='https://api.mypurecloud.com'
    endpoint='/api/v2/authorization/subjects/{0}'.format(row['USERID'])
    api_endpoint = '{0}{1}'.format(base_url, endpoint)
    print(api_endpoint, auth_token)

    # call fetch_data, passing endpoint and auth token
    users_data=fetch_data(api_endpoint, auth_token)

    print(users_data)

# list to collect all data from each pagination api hit
all_data=[]

# log results
print(all_data)
print('Record count: ', len(all_data))

#create df with results
df=pandas.DataFrame({'json_data':all_data})
snowflake_df=session.create_dataframe(df)

# truncate staging table and load data. comment out for first run
# session.sql(f"truncate table {table_name}").collect() 
# snowflake_df write mode "append” will: Append data of this DataFrame to the existing table. Creates a table if it does not exist.
snowflake_df.write.mode("append").save_as_table(table_name)